In [1]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pprint
import pyspark
import pyspark.sql.functions as F

from pyspark.sql.functions import col
from pyspark.sql.types import StringType, IntegerType, FloatType, DateType
import utils.data_processing_bronze_table
import utils.data_processing_silver_table
import utils.data_processing_gold_table

In [2]:
# Initialize SparkSession
spark = pyspark.sql.SparkSession.builder \
    .appName("dev") \
    .master("local[*]") \
    .getOrCreate()

# Set log level to ERROR to hide warnings
spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/16 16:37:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# set up config
snapshot_date_str = "1999-01-01"

start_date_str = "1999-01-01"
end_date_str = "2008-12-31"

In [4]:
# generate list of dates to process
def generate_first_of_month_dates(start_date_str, end_date_str):
    # Convert the date strings to datetime objects
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
    
    # List to store the first of month dates
    first_of_month_dates = []

    # Start from the first of the month of the start_date
    current_date = datetime(start_date.year, start_date.month, 1)

    while current_date <= end_date:
        # Append the date in yyyy-mm-dd format
        first_of_month_dates.append(current_date.strftime("%Y-%m-%d"))
        
        # Move to the first of the next month
        if current_date.month == 12:
            current_date = datetime(current_date.year + 1, 1, 1)
        else:
            current_date = datetime(current_date.year, current_date.month + 1, 1)

    return first_of_month_dates

dates_str_lst = generate_first_of_month_dates(start_date_str, end_date_str)
dates_str_lst

['1999-01-01',
 '1999-02-01',
 '1999-03-01',
 '1999-04-01',
 '1999-05-01',
 '1999-06-01',
 '1999-07-01',
 '1999-08-01',
 '1999-09-01',
 '1999-10-01',
 '1999-11-01',
 '1999-12-01',
 '2000-01-01',
 '2000-02-01',
 '2000-03-01',
 '2000-04-01',
 '2000-05-01',
 '2000-06-01',
 '2000-07-01',
 '2000-08-01',
 '2000-09-01',
 '2000-10-01',
 '2000-11-01',
 '2000-12-01',
 '2001-01-01',
 '2001-02-01',
 '2001-03-01',
 '2001-04-01',
 '2001-05-01',
 '2001-06-01',
 '2001-07-01',
 '2001-08-01',
 '2001-09-01',
 '2001-10-01',
 '2001-11-01',
 '2001-12-01',
 '2002-01-01',
 '2002-02-01',
 '2002-03-01',
 '2002-04-01',
 '2002-05-01',
 '2002-06-01',
 '2002-07-01',
 '2002-08-01',
 '2002-09-01',
 '2002-10-01',
 '2002-11-01',
 '2002-12-01',
 '2003-01-01',
 '2003-02-01',
 '2003-03-01',
 '2003-04-01',
 '2003-05-01',
 '2003-06-01',
 '2003-07-01',
 '2003-08-01',
 '2003-09-01',
 '2003-10-01',
 '2003-11-01',
 '2003-12-01',
 '2004-01-01',
 '2004-02-01',
 '2004-03-01',
 '2004-04-01',
 '2004-05-01',
 '2004-06-01',
 '2004-07-

In [5]:
# create bronze datalake
bronze_diabetes_directory = "datamart/bronze/diabetes/"

if not os.path.exists(bronze_diabetes_directory):
    os.makedirs(bronze_diabetes_directory)


In [6]:
# run bronze backfill
for date_str in dates_str_lst:
    utils.data_processing_bronze_table.process_bronze_table_diabetes(date_str, bronze_diabetes_directory, spark)

1999-01-01row count: 868
saved to: datamart/bronze/diabetes/bronze_diabetes_monthly_1999_01.csv
1999-02-01row count: 784
saved to: datamart/bronze/diabetes/bronze_diabetes_monthly_1999_02.csv
1999-03-01row count: 868
saved to: datamart/bronze/diabetes/bronze_diabetes_monthly_1999_03.csv
1999-04-01row count: 840
saved to: datamart/bronze/diabetes/bronze_diabetes_monthly_1999_04.csv
1999-05-01row count: 868
saved to: datamart/bronze/diabetes/bronze_diabetes_monthly_1999_05.csv
1999-06-01row count: 840
saved to: datamart/bronze/diabetes/bronze_diabetes_monthly_1999_06.csv
1999-07-01row count: 868
saved to: datamart/bronze/diabetes/bronze_diabetes_monthly_1999_07.csv
1999-08-01row count: 868
saved to: datamart/bronze/diabetes/bronze_diabetes_monthly_1999_08.csv
1999-09-01row count: 840
saved to: datamart/bronze/diabetes/bronze_diabetes_monthly_1999_09.csv
1999-10-01row count: 868
saved to: datamart/bronze/diabetes/bronze_diabetes_monthly_1999_10.csv
1999-11-01row count: 840
saved to: datam

saved to: datamart/bronze/diabetes/bronze_diabetes_monthly_2006_07.csv


2006-08-01row count: 868
saved to: datamart/bronze/diabetes/bronze_diabetes_monthly_2006_08.csv
2006-09-01row count: 840
saved to: datamart/bronze/diabetes/bronze_diabetes_monthly_2006_09.csv
2006-10-01row count: 868
saved to: datamart/bronze/diabetes/bronze_diabetes_monthly_2006_10.csv
2006-11-01row count: 840
saved to: datamart/bronze/diabetes/bronze_diabetes_monthly_2006_11.csv
2006-12-01row count: 868
saved to: datamart/bronze/diabetes/bronze_diabetes_monthly_2006_12.csv
2007-01-01row count: 868
saved to: datamart/bronze/diabetes/bronze_diabetes_monthly_2007_01.csv
2007-02-01row count: 784
saved to: datamart/bronze/diabetes/bronze_diabetes_monthly_2007_02.csv
2007-03-01row count: 868
saved to: datamart/bronze/diabetes/bronze_diabetes_monthly_2007_03.csv
2007-04-01row count: 840
saved to: datamart/bronze/diabetes/bronze_diabetes_monthly_2007_04.csv
2007-05-01row count: 868
saved to: datamart/bronze/diabetes/bronze_diabetes_monthly_2007_05.csv
2007-06-01row count: 840
saved to: datam

In [6]:
# create silver datalake
silver_diabetes_monthly_directory = "datamart/silver/diabetes/"

if not os.path.exists(silver_diabetes_monthly_directory):
    os.makedirs(silver_diabetes_monthly_directory)


In [7]:
bronze_diabetes_directory = "datamart/bronze/diabetes/"

# run silver backfill
for date_str in dates_str_lst:
    utils.data_processing_silver_table.process_silver_table(date_str, bronze_diabetes_directory,silver_diabetes_monthly_directory, spark)

loaded from: datamart/bronze/diabetes/bronze_diabetes_monthly_1999_01.csv row count: 868


saved to: datamart/silver/diabetes/silver_diabetes_monthly_1999_01.parquet
loaded from: datamart/bronze/diabetes/bronze_diabetes_monthly_1999_02.csv row count: 784


saved to: datamart/silver/diabetes/silver_diabetes_monthly_1999_02.parquet
loaded from: datamart/bronze/diabetes/bronze_diabetes_monthly_1999_03.csv row count: 868
saved to: datamart/silver/diabetes/silver_diabetes_monthly_1999_03.parquet
loaded from: datamart/bronze/diabetes/bronze_diabetes_monthly_1999_04.csv row count: 840


saved to: datamart/silver/diabetes/silver_diabetes_monthly_1999_04.parquet
loaded from: datamart/bronze/diabetes/bronze_diabetes_monthly_1999_05.csv row count: 868


saved to: datamart/silver/diabetes/silver_diabetes_monthly_1999_05.parquet
loaded from: datamart/bronze/diabetes/bronze_diabetes_monthly_1999_06.csv row count: 840


...

saved to: datamart/silver/diabetes/silver_diabetes_monthly_2008_08.parquet
loaded from: datamart/bronze/diabetes/bronze_diabetes_monthly_2008_09.csv row count: 810


saved to: datamart/silver/diabetes/silver_diabetes_monthly_2008_09.parquet
loaded from: datamart/bronze/diabetes/bronze_diabetes_monthly_2008_10.csv row count: 837


saved to: datamart/silver/diabetes/silver_diabetes_monthly_2008_10.parquet
loaded from: datamart/bronze/diabetes/bronze_diabetes_monthly_2008_11.csv row count: 810


saved to: datamart/silver/diabetes/silver_diabetes_monthly_2008_11.parquet
loaded from: datamart/bronze/diabetes/bronze_diabetes_monthly_2008_12.csv row count: 837


saved to: datamart/silver/diabetes/silver_diabetes_monthly_2008_12.parquet


In [8]:
utils.data_processing_silver_table.process_silver_table('1999-01-01', bronze_diabetes_directory,silver_diabetes_monthly_directory, spark).toPandas()

loaded from: datamart/bronze/diabetes/bronze_diabetes_monthly_1999_01.csv row count: 868


saved to: datamart/silver/diabetes/silver_diabetes_monthly_1999_01.parquet


,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,snapshot_date
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,NO,1999-01-01
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,Up,No,No,No,No,No,Ch,Yes,>30,1999-01-02
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,Yes,NO,1999-01-03
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,Up,No,No,No,No,No,Ch,Yes,NO,1999-01-04
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,Steady,No,No,No,No,No,Ch,Yes,NO,1999-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
863,403731164,173412815,AfricanAmerican,Male,[50-60),?,1,1,7,2,...,Down,No,No,No,No,No,Ch,Yes,NO,1999-01-27
864,403731254,141391067,Caucasian,Female,[40-50),?,3,1,1,1,...,No,No,No,No,No,No,No,No,NO,1999-01-28
865,403731698,71050527,Caucasian,Male,[70-80),?,1,3,7,5,...,Steady,No,No,No,No,No,No,Yes,<30,1999-01-29
866,403737938,40942287,Caucasian,Male,[60-70),?,3,1,1,1,...,Up,No,No,No,No,No,Ch,Yes,>30,1999-01-30


In [7]:
# # set dpd label definition
# dpd = 30

# # Path to the folder containing CSV files
# folder_path = silver_loan_daily_directory

# # Read all CSV files into a single DataFrame
# files_list = [folder_path+os.path.basename(f) for f in glob.glob(os.path.join(folder_path, '*'))]
# df = spark.read.option("header", "true").parquet(*files_list)

# # filter only completed loans
# df = df.filter(col("loan_start_date") < datetime.strptime("2024-01-01", "%Y-%m-%d"))

# # create dpd flag if more than dpd
# df = df.withColumn("dpd_flag", F.when(col("dpd") >= dpd, 1).otherwise(0))

# # actual bads 
# actual_bads_df = df.filter(col("installment_num") == 10)

# # prepare for analysis
# # df = df.filter(col("installment_num") < 10)

# # visualise bad rate
# pdf = df.toPandas()

# # Group by col_A and count occurrences in col_B
# grouped = pdf.groupby('mob')['dpd_flag'].mean()

# # Sort the index (x-axis) of the grouped DataFrame
# grouped = grouped.sort_index()

# # Plotting
# grouped.plot(kind='line', marker='o')

# plt.title('DPD: '+ str(dpd))
# plt.xlabel('mob')
# plt.ylabel('bad rate')
# plt.grid(True)
# plt.show()

In [8]:
# create bronze datalake
gold_label_store_directory = "datamart/gold/label_store/"
gold_feature_store_directory = "datamart/gold/feature_store/"

if not os.path.exists(gold_label_store_directory) and not os.path.exists(gold_feature_store_directory):
    os.makedirs(gold_label_store_directory)
    os.makedirs(gold_feature_store_directory)

In [9]:
# run gold backfill
for date_str in dates_str_lst:
    utils.data_processing_gold_table.process_labels_gold_table(date_str, silver_diabetes_monthly_directory, gold_label_store_directory, spark)
    utils.data_processing_gold_table.process_features_gold_table(date_str, silver_diabetes_monthly_directory, gold_feature_store_directory, spark)

loaded from: datamart/silver/diabetes/silver_diabetes_monthly_1999_01.parquet row count: 868


saved to: datamart/gold/label_store/gold_label_store_1999_01.parquet
loaded from: datamart/silver/diabetes/silver_diabetes_monthly_1999_01.parquet row count: 868


/usr/local/lib/python3.12/site-packages/pyspark/sql/column.py:460: FutureWarning: A column as 'key' in getItem is deprecated as of Spark 3.0, and will not be supported in the future release. Use `column[key]` or `column.key` syntax instead.
  warnings.warn(


saved to: datamart/gold/feature_store/gold_feature_store_1999_01.parquet
loaded from: datamart/silver/diabetes/silver_diabetes_monthly_1999_02.parquet row count: 784
saved to: datamart/gold/label_store/gold_label_store_1999_02.parquet
loaded from: datamart/silver/diabetes/silver_diabetes_monthly_1999_02.parquet row count: 784
saved to: datamart/gold/feature_store/gold_feature_store_1999_02.parquet
loaded from: datamart/silver/diabetes/silver_diabetes_monthly_1999_03.parquet row count: 868


saved to: datamart/gold/label_store/gold_label_store_1999_03.parquet
loaded from: datamart/silver/diabetes/silver_diabetes_monthly_1999_03.parquet row count: 868


...

saved to: datamart/gold/feature_store/gold_feature_store_2008_10.parquet
loaded from: datamart/silver/diabetes/silver_diabetes_monthly_2008_11.parquet row count: 810


saved to: datamart/gold/label_store/gold_label_store_2008_11.parquet
loaded from: datamart/silver/diabetes/silver_diabetes_monthly_2008_11.parquet row count: 810


saved to: datamart/gold/feature_store/gold_feature_store_2008_11.parquet
loaded from: datamart/silver/diabetes/silver_diabetes_monthly_2008_12.parquet row count: 837


saved to: datamart/gold/label_store/gold_label_store_2008_12.parquet
loaded from: datamart/silver/diabetes/silver_diabetes_monthly_2008_12.parquet row count: 837


saved to: datamart/gold/feature_store/gold_feature_store_2008_12.parquet


In [10]:
folder_path = gold_label_store_directory
files_list = [folder_path+os.path.basename(f) for f in glob.glob(os.path.join(folder_path, '*'))]
df = spark.read.option("header", "true").parquet(*files_list)
print("row_count:",df.count())

df.show()

row_count: 101766
+------------+-----+-------------+
|encounter_id|label|snapshot_date|
+------------+-----+-------------+
|     4679262|    0|   2000-08-01|
|     4680186|    0|   2000-08-02|
|     4682388|    0|   2000-08-03|
|     4728330|    0|   2000-08-04|
|     4729674|    0|   2000-08-05|
|     4730190|    0|   2000-08-06|
|     4731546|    0|   2000-08-07|
|     4734798|    0|   2000-08-08|
|     4736166|    0|   2000-08-09|
|     4736214|    0|   2000-08-10|
|     4765758|    0|   2000-08-11|
|     4772286|    0|   2000-08-12|
|     4798818|    0|   2000-08-13|
|     4801326|    0|   2000-08-14|
|     4804620|    1|   2000-08-15|
|     4804968|    0|   2000-08-16|
|     4809204|    0|   2000-08-17|
|     4834632|    0|   2000-08-18|
|     4840590|    0|   2000-08-19|
|     4840836|    0|   2000-08-20|
+------------+-----+-------------+
only showing top 20 rows



In [11]:
folder_path = gold_feature_store_directory
files_list = [folder_path+os.path.basename(f) for f in glob.glob(os.path.join(folder_path, '*'))]
df = spark.read.option("header", "true").parquet(*files_list)
print("row_count:",df.count())

df.show()

row_count: 101766
+------------+--------------------+--------------+----------+-------------+---------+------------+------------------------+---------------------------+--------------------+-------------+-----------+-----------+-----------------+--------------------+------+------+------+-------------+
|encounter_id|race_AfricanAmerican|race_Caucasian|race_Asian|race_Hispanic|is_female|age_midpoint|admission_severity_score|admission_source_risk_score|poor_glucose_control|metformin_ord|insulin_ord|diabetesMed|severity_x_visits|  medication_density|diag_1|diag_2|diag_3|snapshot_date|
+------------+--------------------+--------------+----------+-------------+---------+------------+------------------------+---------------------------+--------------------+-------------+-----------+-----------+-----------------+--------------------+------+------+------+-------------+
|    15758256|                   0|             0|         1|            0|        0|          65|                       1|    

In [12]:
df.toPandas().to_csv('gold_feature_store.csv', index=False)

EDA

In [13]:
# Read label store
folder_path = gold_label_store_directory
files_list = [folder_path + os.path.basename(f) for f in glob.glob(os.path.join(folder_path, '*'))]
label_df = spark.read.option("header", "true").parquet(*files_list)
print("Label DF row_count:", label_df.count())

# Read feature store
folder_path = gold_feature_store_directory
files_list = [folder_path + os.path.basename(f) for f in glob.glob(os.path.join(folder_path, '*'))]
feature_df = spark.read.option("header", "true").parquet(*files_list)
print("Feature DF row_count:", feature_df.count())

# --- Join both DataFrames on encounter_id and snapshot_date ---
final_df = feature_df.join(
    label_df,
    on=["encounter_id", "snapshot_date"],
    how="inner"  # can be 'left', 'right', or 'outer' depending on your need
)

# Show results
print("Final DF row_count:", final_df.count())
final_df.show(truncate=False)

Label DF row_count: 101766


Feature DF row_count: 101766


Final DF row_count: 101766


+------------+-------------+--------------------+--------------+----------+-------------+---------+------------+------------------------+---------------------------+--------------------+-------------+-----------+-----------+-----------------+--------------------+------+------+------+-----+
|encounter_id|snapshot_date|race_AfricanAmerican|race_Caucasian|race_Asian|race_Hispanic|is_female|age_midpoint|admission_severity_score|admission_source_risk_score|poor_glucose_control|metformin_ord|insulin_ord|diabetesMed|severity_x_visits|medication_density  |diag_1|diag_2|diag_3|label|
+------------+-------------+--------------------+--------------+----------+-------------+---------+------------+------------------------+---------------------------+--------------------+-------------+-----------+-----------+-----------------+--------------------+------+------+------+-----+
|15758256    |2005-08-01   |0                   |0             |1         |0            |0        |65          |1              

In [14]:
target_col = "label"  # or your actual label column name
feature_cols = [c for c in final_df.columns if c not in ["encounter_id", "snapshot_date", target_col, "medical_specialty", 'diag_1', 'diag_2', 'diag_3']]


In [15]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
vector_df = assembler.transform(final_df).select("features", target_col)


In [16]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

# Convert label to numeric if needed
indexer = StringIndexer(inputCol=target_col)

rf = RandomForestClassifier(
    featuresCol="features",
    labelCol="label",
    numTrees=100,
    maxDepth=10,
    seed=42
)

pipeline = Pipeline(stages=[indexer, assembler, rf])
model = pipeline.fit(final_df)

rf_model = model.stages[-1]  # extract trained RF model


In [17]:
import pandas as pd

importances = rf_model.featureImportances.toArray()
feature_importance_df = pd.DataFrame({
    "feature": feature_cols,
    "importance": importances
}).sort_values(by="importance", ascending=False)

feature_importance_df.head(20)


,feature,importance
12,severity_x_visits,0.473083
13,medication_density,0.136300
5,age_midpoint,0.112098
10,insulin_ord,0.045787
7,admission_source_risk_score,0.043072
6,admission_severity_score,0.042764
9,metformin_ord,0.035358
4,is_female,0.022178
8,poor_glucose_control,0.021805
11,diabetesMed,0.017363
